# The Missing Semester: Git

Se trata de un sistema de control de versiones, permite mantener un historial sobre los cambios de un archivo (asi como el perpetrador de cada cambio) y facilita la colaboracion.

**Nota:** Este notebook asume que el lector es familiar con los terminos de Git: branch, commit, merge y repositorio

Git esta basado en arboles dirigidos y arboles, estos ultimos son una variante particular de grafos no dirigidos en la cual dos vertices se conectan por *exactamente un* camino o arista (es decir, se trata de un grafo sin ciclos). Vale la pena recalcar que un bosque es un grafo no dirigido cuyos vertices estan conectados por *a lo sumo o maximo* un camino o arista, es decir que un bosque puede ser visto como una coleccion de arboles que pueden estar desconectados o conectados entre si por a lo sumo un vertice. Un arbol dirigido es un grafo dirigido que resulta siendo un arbol al remover la direccion de sus aristas.

Un arbol dirigido:
![alt text](polytree.png "Arbol dirigido")

Es importante mencionar esto porque, a pesar de que Git esta basadso en arboles dirigidos de teoria de grafos, en Git tambien se usa el termino de "arbol" para referirse a directorios dentro de su modelo de datos.

# Modelo de datos

## Snapshots

Como se menciono anteriormente, git utiliza un modelo de datos en el cual se le llama arbol a un directorio (una carpeta) y se denomina blob a cualquier archivo dentro de un arbol. Un arbol realmente sirve para mapear nombres a blobs y a otros arboles, de tal manera que un arbol puede contener a mas arboles. Por ejemplo:

    <root> (tree)  
    |  
    +- foo (tree)  
    |  |  
    |  + bar.txt (blob, contents = "hello world")  
    |  
    +- baz.txt (blob, contents = "git is wonderful")  
    
Se puede apreciar como una carpeta llamada root es el arbol principal, el cual contiene al arbol foo y al blog baz.txt. Dentro del arbol foo podemos encontrar el archivo bar.txt.


Sin embargo, el arbol raiz, en el ejemplo anterior es literalmente el arbol llamado root, es de particular mencion porque es el arbol sobre el cual se hace el control de versionamiento (es decir, este arbol y sus contenidos, ignorando a sus padres y a los demas arboles que se encuentran en el mismo nivel, son los que rastrea el sistema de versiones de Git) y estos arboles son denominados *snapshots* (pueden imaginarse como fotos).

Cuando se hace un *commit* de Git, realmente lo que se esta haciendo es crear un snapshot, diciendole a Git "toma una foto a mis archivos en este preciso instante".

## Arbol dirigido como historial de snapshots

Los snapshots son una coleccion de arboles y blobs bajo un solo arbol principal, pero esto es basicamente cualquier carpeta dentro de un sistema de archivos, lo que hace a Git un sistema de version de controles es que git guarda un snapshot y lo relaciona con un otro snapshot que representa el estado previo de los archivos, de tal manera que para una serie de snapshots, se crea un arbol dirigido (de teoria de grafos) en donde cada vertice es un snapshot y la direccion de cada arista apunta al vertice directamente anterior en el tiempo, por ejemplo:

        o <-- o <-- o <-- o
        Notar que cada o es un vertice (snapshot) en un arbol dirigido

El `o` que se encuentra mas a la derecha es el ultimo snapshot, o commit, en una serie de cuatro snapshots, recordando siempre que la flecha apunta a su predecesor directo (es decir, la flecha apunta hacia *atras* en el tiempo).

Claro, el ejemplo anterior es sencillo porque se trata de una serie de snapshots lineales, pero en git existen *branches*, permitiendo que un snapshot tenga mas de un solo predecesor en el tiempo, de la siguiente manera:

    o <-- o <-- o <-- o <---- o
                ^            /
                 \          v
                  --- o <-- o
              
Siendo esto la representacion de un *merge* entre dos branches, donde el `o` mas a la derecha es el mas reciente y es el descendiente directo de dos vertices distintos.

## Objetos, content-addresing y referencias

Para Git, cada blob, arbol o snapshot es un *objeto*. Cada objeto en Git es direccionado por su contenido (content-addressed). Esto quiere decir que a cada objeto se le asigna una direccion unica definida por su SHA-1 (Secure Hash Algorithm) Hash segun su contenido y luego almacena un mapeo de archivo a SHA-1 Hash. Esto tiene una serie de ventajas como:

1. Deteccion de duplicados: Un archivo puede ser copiado multiples veces en un solo snapshot, a Git no le importa porque no almacena el archivo tres veces, simplemente hace referencia al mismo hash tres veces. Cuando un archivo cambia de arbol (pasa de una carpeta a otra), su SHA-1 no cambia, por lo que Git puede darse cuenta del movimiento sin rescribir nada, simplemente apuntando el hash al lugar correcto. La eliminacion de un archivo en un snapshot y la creacion de un archivo identico no genera mas overhead de almacenamiento en Git, debido a que el archivo ya se conoce y se puede encontrar en un snapshot previo.
1. Se encuentran diferencias o cambios rapidamente, es suficiente con verificar que el SHA-1 no sea el mismo para dos objetos de mismo nombre para darse cuenta de que estos objetos son distintos.

Git, teniendo una manera sencilla para identificar a cada objeto a traves de su SHA-1 Hash, tambien puede identificar a cada snapshot bajo algun nombre conveniente como "master", "main" o "cool_new_feature", de tal manera que un humano sea capaz de leer y entender en que snapshot se encuentra trabajando (este snapshot en particular, el "activo" se denomina HEAD), esta asignacion de hash a snapshot se denomina referencia.

### Repositorios

Un repositorio es la coleccion de objetos y referencias sobre las cuales Git ha estado rastreando cambios. Un usuario que interactua con Git interactua con el repositorio de Git, es decir que realmente la utilizacion de comandos como "commit" o "add" se transforman en una actualizacion de los objetos y referencias del repositorio y Git mantiene el control de estos cambios.

## Staging Area
Algo particularmente util de Git es la presencia de la denominada Staging Area. Realmente esta sirve como una especie de "carrito de compras" de cambios, es decir que un usuario puede, por ejemplo, seleccionar que partes especificas del directorio de trabaja seran almacenadas durante el commit (la toma de la foto, o el snapshot) y dejar el resto de cambios como impercibibles en la foto. La utilidad de esto es que permite que un usuario, por necesidad o cualquier otro motivo, trabaje en dos features simultaneamente y que, a la hora de hacer un commit, no tenga que hacer un solo commit para ambas features sino que puede separar y elegir a mano que cambios especificamente son los que entraran en un commit, ignorando el resto. De esta manera un desarrollador puede separar de manera mas limpia la implementacion de dos o mas features en cuantos commits requiera.

# Ejemplos

El comando git help nos da una lista de comandos utilies y sus funciones

In [1]:
!!git

['usage: git [--version] [--help] [-C <path>] [-c <name>=<value>]',
 '           [--exec-path[=<path>]] [--html-path] [--man-path] [--info-path]',
 '           [-p | --paginate | -P | --no-pager] [--no-replace-objects] [--bare]',
 '           [--git-dir=<path>] [--work-tree=<path>] [--namespace=<name>]',
 '           [--super-prefix=<path>] [--config-env=<name>=<envvar>]',
 '           <command> [<args>]',
 '',
 'These are common Git commands used in various situations:',
 '',
 'start a working area (see also: git help tutorial)',
 '   clone     Clone a repository into a new directory',
 '   init      Create an empty Git repository or reinitialize an existing one',
 '',
 'work on the current change (see also: git help everyday)',
 '   add       Add file contents to the index',
 '   mv        Move or rename a file, a directory, or a symlink',
 '   restore   Restore working tree files',
 '   rm        Remove files from the working tree and from the index',
 '',
 'examine the history and 

Los branches son particularmente utiles para desarrollar features en paralelo o cuando se esta colaborando con mas personas porque los cambios son independientes. Veamos que branches existen en este repositorio:

In [2]:
!git branch

* master
  prueba


Podemos ver que existen dos branches, master y prueba. El asterisco al lado de master indica que el branch actual es master, veamos sus contenidos:

In [3]:
!git ls-files

.gitignore
Git.ipynb
polytree.png


Unicamente dos archivos, este mismo notebook y una imagen, cambiemos al otro branch y veamos que contiene

In [4]:
!git checkout prueba

M	Git.ipynb
Your branch is up to date with 'origin/prueba'.


Switched to branch 'prueba'


In [5]:
!git branch

  master
* prueba


In [6]:
!git ls-files

Git.ipynb
hola mundo.txt
polytree.png


Se puede notar que el branch prueba contiene un archivo adicional, "hola mundo.txt", el cual no existe en el branch master. Un ejemplo de como los cambios de un branch no afectan al otro.

Seria interesante ver que branch es el que tiene cambios mas recientes:

In [7]:
!git log --all --graph --decorate

* commit 2467b937ae27dfc40d4c7b9148b2dd851b9e8074 (origin/master, master)
| Author: H0CHM31573R <yancosr@rocketmail.com>
| Date:   Tue Feb 8 20:32:57 2022 -0600
| 
|     added gitignore
|   
| * commit 65d6b6641afc236ad3a64e810a18e72202372f4c (HEAD -> prueba, origin/prueba)
|/  Author: H0CHM31573R <yancosr@rocketmail.com>
|   Date:   Tue Feb 8 20:17:49 2022 -0600
|   
|       Added hola mundo text file to exemplify branches
| 
* commit ef76c5841ab08d797facd62e2998989fbbe71b5f
  Author: H0CHM31573R <yancosr@rocketmail.com>
  Date:   Tue Feb 8 19:55:11 2022 -0600
  
      Resumen creado


Regresemos a master

In [8]:
!git checkout master

M	Git.ipynb
Your branch is up to date with 'origin/master'.


Switched to branch 'master'


Realicemos un cambio al branch

In [9]:
!echo "un terrible error" > cool_feature.txt

In [10]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Git.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	cool_feature.txt

no changes added to commit (use "git add" and/or "git commit -a")


Podemos observar que nuestro cool_feature.txt fue detectado como un cambio, asi que lo agregaremos a nuestro branch master

In [11]:
!git add cool_feature.txt

In [12]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes to be committed:
  (use "git restore --staged <file>..." to unstage)
	new file:   cool_feature.txt

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Git.ipynb



Podemos ver que el archivo cool_feature.txt sera incluido en un commit, asi que hagamos un commit

In [13]:
!git commit -m "Adding a cool new feature"

[master 36b8fcf] Adding a cool new feature
 1 file changed, 1 insertion(+)
 create mode 100644 cool_feature.txt


In [14]:
!git status

On branch master
Your branch is ahead of 'origin/master' by 1 commit.
  (use "git push" to publish your local commits)

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Git.ipynb

no changes added to commit (use "git add" and/or "git commit -a")


Sucede que justo en este momento hemos recibido una llamada, la implementacion de cool new feature no solamente es innecesaria sino que es indeseada, por suerte podemos resolver este problema al resetear el staging area

In [15]:
!git reset HEAD~

Unstaged changes after reset:
M	Git.ipynb


In [16]:
!git status

On branch master
Your branch is up to date with 'origin/master'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
	modified:   Git.ipynb

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	cool_feature.txt

no changes added to commit (use "git add" and/or "git commit -a")


Podemos ver que cool_feature ya no esta siendo tomado en cuenta, basicamente acabamos de resetear el staging area al estado de HEAD.

En retrospectiva nos podemos dar cuenta de que implementar cool_feature hubiese sido un terrible error, de hecho es algo vergonzoso, asi que eliminaremos la evidencia:

In [17]:
import os
os.remove("cool_feature.txt")

Listo!